---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

def blight_model():
    # Your code here
    df_train = pd.read_csv('train.csv', encoding='latin')
    df_train.drop(['grafitti_status', 'violation_zip_code', 'clean_up_cost',
                   'fine_amount', 'admin_fee', 'state_fee','late_fee',
                   'discount_amount', 'violation_code', 'non_us_str_code',
                   'violation_street_number', 'mailing_address_str_number', 'zip_code',
                   'agency_name', 'inspector_name',
                   'payment_amount', 'balance_due', 'payment_date', 'payment_status', 
                   'collection_status', 'compliance_detail', 'mailing_address_str_name',
                   'violation_street_name', 'judgment_amount'], 1, inplace=True)
    df_train.dropna(subset=['compliance'], inplace=True)
    df_train.replace({'collection_status':'IN COLLECTION'}, True, inplace=True)
    df_train.fillna({'collection_status': False}, inplace=True)
    df_train.set_index('ticket_id', inplace=True)
    
    df_test = pd.read_csv('test.csv')
    df_test.drop(['grafitti_status', 'violation_zip_code', 'clean_up_cost',
                  'fine_amount', 'admin_fee', 'state_fee', 'late_fee',
                  'discount_amount', 'violation_code', 'non_us_str_code',
                  'violation_street_number', 'mailing_address_str_number', 'zip_code',
                  'agency_name', 'inspector_name', 'mailing_address_str_name',
                  'violation_street_name', 'judgment_amount'], 1, inplace=True)
    df_test.set_index('ticket_id', inplace=True)
    
    df_train.drop(['ticket_issued_date', 'hearing_date'], 1, inplace=True)#.astype('datetime64')
    df_test.drop(['ticket_issued_date', 'hearing_date'], 1, inplace=True)#.astype('datetime64')
    
    for label_name in ['disposition', 'violation_description', 'violator_name',
                      'city', 'state', 'country']:
        le = LabelEncoder().fit(df_train[label_name].append(df_test[label_name], ignore_index=True).astype(str))
        df_train[label_name] = le.transform(df_train[label_name].astype(str))
        df_test[label_name] = le.transform(df_test[label_name].astype(str))

    X_train, y_train, X_test = (df_train[df_train.columns[:-1]],
                                       df_train[df_train.columns[-1]],
                                       df_test)
    clf = RandomForestRegressor(n_estimators = 100, max_features = 5, max_depth = 7,
                                random_state=1, warm_start=True).fit(X_train, y_train)
    #param = {'n_estimators': [10, 50, 70, 90, 100, 120]}
    #grid = GridSearchCV(clf, param).fit(X_train, y_train)
    df_test['Probab'] = clf.predict(X_test)
    return df_test['Probab']#grid.best_params_
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.176401
285362    0.035573
285361    0.057024
285338    0.039314
285346    0.080486
285345    0.036339
285347    0.061863
285342    0.471345
285530    0.036232
284989    0.036329
285344    0.056073
285343    0.040211
285340    0.035477
285341    0.046519
285349    0.122783
285348    0.037937
284991    0.036329
285532    0.035573
285406    0.035584
285001    0.037560
285006    0.048213
285405    0.036369
285337    0.036329
285496    0.047371
285497    0.036329
285378    0.036232
285589    0.036329
285585    0.035573
285501    0.056785
285581    0.035573
            ...   
376367    0.045455
376366    0.037316
376362    0.271654
376363    0.222300
376365    0.045442
376364    0.035684
376228    0.035927
376265    0.036017
376286    0.293119
376320    0.036897
376314    0.037316
376327    0.336705
376385    0.337696
376435    0.459929
376370    0.268170
376434    0.059773
376459    0.056110
376478    0.032721
376473    0.036004
376484    0.028112
376482    0.023074
37

In [10]:
df_train = pd.read_csv('train.csv', encoding='latin')
df_test = pd.read_csv('test.csv', encoding='latin')

df_train.drop(['grafitti_status', 'violation_zip_code', 'clean_up_cost',
               'fine_amount', 'admin_fee', 'state_fee','late_fee',
               'discount_amount', 'violation_code', 'non_us_str_code',
               'violation_street_number', 'mailing_address_str_number', 'zip_code',
               'agency_name', 'inspector_name',
               'payment_amount', 'balance_due', 'payment_date', 'payment_status', 
               'collection_status', 'compliance_detail', 'mailing_address_str_name',
               'violation_street_name'], 1, inplace=True)
df_train.dropna(subset=['compliance'], inplace=True)
df_train.replace({'collection_status':'IN COLLECTION'}, True, inplace=True)
df_train.fillna({'collection_status': False}, inplace=True)
df_train.set_index('ticket_id', inplace=True)

df_train.drop(['ticket_issued_date', 'hearing_date'], 1, inplace=True)
df_test.drop(['ticket_issued_date', 'hearing_date'], 1, inplace=True)


df_test.drop(['grafitti_status', 'violation_zip_code', 'clean_up_cost',
                  'fine_amount', 'admin_fee', 'state_fee', 'late_fee',
                  'discount_amount', 'violation_code', 'non_us_str_code',
                  'violation_street_number', 'mailing_address_str_number', 'zip_code',
                  'agency_name', 'inspector_name', 'mailing_address_str_name',
                  'violation_street_name'], 1, inplace=True)
df_test.set_index('ticket_id', inplace=True)

df_test

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,violator_name,city,state,country,violation_description,disposition,judgment_amount
ticket_id,,,,,,,
284932,"FLUELLEN, JOHN A",DETROIT,MI,USA,Failure to secure City or Private solid waste ...,Responsible by Default,250.0
285362,"WHIGHAM, THELMA",DETROIT,MI,USA,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1130.0
285361,"WHIGHAM, THELMA",DETROIT,MI,USA,Improper placement of Courville container betw...,Responsible by Default,140.0
285338,"HARABEDIEN, POPKIN",WOODHAVEN,MI,USA,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,250.0
285346,"CORBELL, STANLEY",LIVONIA,MI,USA,Violation of time limit for approved container...,Responsible by Default,140.0
285345,"CORBELL, STANLEY",LIVONIA,MI,USA,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,250.0
285347,"CORBELL, STANLEY",LIVONIA,MI,USA,Inoperable motor vehicle(s) one- or two-family...,Responsible by Default,85.0
285342,"NICKOLA CORPORATION, W & H",ROCHESTER HILLS,MI,USA,Allowing bulk solid waste to lie or accumulate...,Responsible by Determination,230.0
285530,"INTERSTATE INVESTMENT GROUP LL, .",GILBERT,SC,USA,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1130.0
